In [1]:
    try:
        from rites.rites import update_hooks, Partial, AST, Code
    except:
        from rites import update_hooks, Partial, AST, Code
    
    from IPython import get_ipython
    from nbconvert.exporters.templateexporter import TemplateExporter
    from IPython.core.inputtransformer import InputTransformer
    from jinja2 import Environment
    from collections import UserList
    from dataclasses import dataclass, field

    @dataclass
    class Template(InputTransformer, UserList):
        environment = Environment()
        user_ns: dict = field(default=None, repr=False)
        data: list = field(default_factory=list)
        
        def __post_init__(self): 
            if self.user_ns is None:
                self.user_ns = get_ipython().user_ns
            
        def push(self, line): self.data.append(line)
        
        def reset(self, *, str=""""""):
            while self.data: str += self.data.pop(0) + "\n"
            return self.environment.from_string(str).render(**self.user_ns)
        
        def __enter__(self,): ...
        
        def __exit__(self, *args): 
            get_ipython().input_transformer_manager.python_line_transforms = list(
                filter(self.__ne__, get_ipython().input_transformer_manager.python_line_transforms))

TemplateLoader has to execute its own module and cache it in exec module.  Compile does the caching.  Compile needs to be called after the ast is evaluated.

In [2]:
    from types import ModuleType

In [3]:
    class Incremental(AST):
        def from_code_cell(Incremental, cell, **dict):
            module = dict.pop('module')
            Module = super(type(Incremental), Incremental).from_code_cell(cell, **dict)
            if Module:
                eval(Incremental.compile(Module), module.__dict__, module.__dict__)
            return Module
        
        def from_notebook_node(AST, nb, resource: dict=None, **dict):         
            ast = super().from_notebook_node(nb, resource, **dict)
            return AST.compile(ast) and ast 

In [4]:
    class _TemplateLoaderBase(Partial):
        def exec_module(Loader, module):
            with Template(user_ns=module.__dict__):
                parser = Incremental(Loader.path, Loader.name)
                with __import__('io').BytesIO(Loader.get_data(Loader.path)) as data:
                    parser.compile(parser.from_file(data, module=module))
            return module
        
    class TemplateLoader(_TemplateLoaderBase):
        exec_module = Partial.exec_module

In [5]:
    class PartialTemplate(TemplateLoader):
        exec_module = Partial.exec_module

In [6]:
    def load_ipython_extension(ip=get_ipython()):
        ip.input_transformer_manager.python_line_transforms.insert(0, Template())
        update_hooks()
        update_hooks(TemplateLoader)
        
    def unload_ipython_extension(ip=None):
        update_hooks()

In [7]:
    if __name__ == '__main__':
        load_ipython_extension()